In [1]:
# !pip install transformers datasets torch trl peft bitsandbytes

In [3]:
# Load required libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

def generate_text(prompt, max_length=500, temperature=0.1):
    """
    Generate text using the DeepSeek model

    Args:
        prompt (str): Input text to generate from
        max_length (int): Maximum length of generated text
        temperature (float): Controls randomness in generation (0.0-1.0)

    Returns:
        str: Generated text
    """
    # Encode the input text
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1
        )

    # Decode and return the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Example usage
if __name__ == "__main__":
    # Example prompts to test the model
    prompts = [

        "In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)?",
        "Tell me about buddhism in India?"

    ]

    print("Generating text from different prompts:\n")
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        generated = generate_text(prompt)
        print(f"Generated text: {generated}\n")

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating text from different prompts:

Prompt: In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)?
Generated text: In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does it relate to the concept of self-awareness? Can you explain the relationship between these two concepts?

In the context of the yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does it relate to the concept of self-awareness? Can you explain the relationship between these two concepts?

In the context of the yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does it relate to the concept of self-awareness? Can you explain the relationship between these two concepts?

In the context of the yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does it relate to the concept of self-awareness? Can you explain the relationship between these two

In [4]:
%cd

/root


In [5]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from trl import SFTTrainer
import torch
from peft import LoraConfig, get_peft_model

# Step 1: Load the dataset
dataset = load_dataset("Abhaykoul/Ancient-Indian-Wisdom")

# Step 2: Format the dataset into instruction-response pairs
def format_dataset(examples):
    """Format the dataset into instruction-response pairs."""
    texts = []
    for instruction, response in zip(examples["instruction"], examples["output"]):
        # Combine instruction and response into a single text
        formatted_text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
        texts.append(formatted_text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(format_dataset, batched=True, remove_columns=dataset["train"].column_names)

# Step 3: Load model and tokenizer
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Step 4: Configure LoRA
peft_config = LoraConfig(
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # No bias for LoRA
    task_type="CAUSAL_LM",  # Task type
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Target modules for LoRA
)
model = get_peft_model(model, peft_config)

# Step 5: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save results
    num_train_epochs=20,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per device
    per_device_eval_batch_size=4,  # Evaluation batch size
    gradient_accumulation_steps=4,  # Gradient accumulation steps
    gradient_checkpointing=False,  # Disable gradient checkpointing for debugging
    optim="adamw_torch",  # Optimizer
    learning_rate=1e-4,  # Learning rate
    warmup_ratio=0.1,  # Warmup ratio
    fp16=True,  # Use mixed precision (FP16)
    logging_steps=10,  # Log every 10 steps
    save_strategy="steps",  # Save model at specific steps
    save_steps=100,  # Save every 100 steps
    evaluation_strategy="steps",  # Evaluate at specific steps
    eval_steps=100,  # Evaluate every 100 steps
    eval_accumulation_steps=1,  # Accumulate evaluation steps
    load_best_model_at_end=True,  # Load the best model at the end
    metric_for_best_model="eval_loss",  # Metric for best model
    greater_is_better=False,  # Lower eval_loss is better
    remove_unused_columns=True,  # Remove unused columns
    report_to="none",  # Disable external logging
)

# Step 6: Initialize the trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"].select(range(120)),  # Small evaluation set
)


# Step 7: Train the model
trainer.train()



README.md:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

dataset.json:   0%|          | 0.00/718k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/616 [00:00<?, ? examples/s]

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Tokenizing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
100,1.713900,1.754875
200,1.445800,1.540128
300,1.341200,1.412575
400,1.192400,1.325888
500,1.147600,1.265335
600,1.104600,1.224926
700,1.095200,1.201578


TrainOutput(global_step=760, training_loss=1.3439436699214735, metrics={'train_runtime': 2017.69, 'train_samples_per_second': 6.106, 'train_steps_per_second': 0.377, 'total_flos': 3.5950886459904e+16, 'train_loss': 1.3439436699214735})

In [6]:
model.save_pretrained("fine-tuned-deepseek-r1-1.5b")
tokenizer.save_pretrained("fine-tuned-deepseek-r1-1.5b")

('fine-tuned-deepseek-r1-1.5b/tokenizer_config.json',
 'fine-tuned-deepseek-r1-1.5b/special_tokens_map.json',
 'fine-tuned-deepseek-r1-1.5b/tokenizer.json')

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define model path
model_path = "fine-tuned-deepseek-r1-1.5b"

# Load model with FP16 precision and automatic device allocation
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    # device_map="auto"  # Remove or change to a specific device if known
    # Instead of "auto", try specifying the device explicitly if you know which device to use (e.g., "cuda:0")
    device_map='cuda:0' if torch.cuda.is_available() else 'cpu' # Example: Use GPU if available, otherwise CPU
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Function to generate text
def generate_text(prompt, max_new_tokens=1000):
    # Ensure inputs are correctly formatted and placed on the same device as the model
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    # Set seed for reproducibility (optional)
    torch.manual_seed(42)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_k=50,
            top_p=0.9,
            use_cache=True
        )

    # Decode and return the generated text
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test the function
prompt = "In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)?"
output = generate_text(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does this concept relate to the concept of non-violence in lawlessness and compassion?

In Hinduism, the concept of ahimsa is central to the practice of non-violence. It encompasses the practice of compassion, empathy, and non-gainful action towards all living beings, including the non-human. Ahimsa is often referred to as non-violence, lawlessness, and compassion.

In Yoga, ahimsa is the practice of non-violence, which is deeply connected to the concept of non-violence in lawlessness and compassion. The goal is to practice non-violence in all aspects of life, including in the conduct of one's self, relationships, and the actions of others.

In the context of the Bhagavad Gita, ahimsa is a central concept, as it is the foundation of the self-realization and liberation from the cycle of birth and death (samsara).

Ahimsa is not limited to the physical actions of an individual but extends to all asp